In [1]:
from pathlib import Path
from sqlalchemy import create_engine

db_path = Path("data/contractiq.db")
engine = create_engine(f"sqlite:///{db_path}")
print("DB:", db_path.resolve())


DB: /mnt/c/Ubuntu/my_github_repos/ContractIQ/notebooks/data/contractiq.db


In [9]:
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_columns("documents")

[{'name': 'doc_id',
  'type': VARCHAR(),
  'nullable': False,
  'default': None,
  'primary_key': 1},
 {'name': 'title',
  'type': TEXT(),
  'nullable': False,
  'default': None,
  'primary_key': 0},
 {'name': 'source',
  'type': VARCHAR(),
  'nullable': False,
  'default': None,
  'primary_key': 0},
 {'name': 'text',
  'type': TEXT(),
  'nullable': False,
  'default': None,
  'primary_key': 0}]

In [13]:
from sqlalchemy import select, func, text

with engine.connect() as conn:
    doc_count = conn.execute(text("SELECT doc_id FROM documents"))

# print(f"doc_count: {doc_count}")
for r in doc_count:
    print(r)

('2d790a4a4132cc53',)
('43ab152a17a15599',)
('644b67c819fbca9c',)
('b26c38227ce11a4e',)
('b9d11c50bada2ade',)
('f71b4149a185d016',)


In [16]:
with engine.connect() as conn:
    num_docs = conn.execute(text("SELECT COUNT(*) FROM documents")).scalar_one()
    num_chunks = conn.execute(text("SELECT COUNT(*) FROM chunks")).scalar_one()
    num_anns = conn.execute(text("SELECT COUNT(*) FROM annotations")).scalar_one()

print(f"num_docs: {num_docs}, num_chunks: {num_chunks}, num_anns: {num_anns}")

num_docs: 6, num_chunks: 224, num_anns: 312


### Gemini Embeddings (we are not using this due to API call limits for free tier)

In [19]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

load_dotenv(override=True)
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY=os.getenv("GEMINI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY


In [21]:
embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")  # :contentReference[oaicite:2]{index=2}
vec_1= embeddings.embed_query("dimension check")
vec_2= embeddings.embed_query("another random query")

EMBED_DIM_1 = len(vec_1)

print("Embedding dimension 1:", EMBED_DIM_1)

EMBED_DIM_2 = len(vec_2)

print("Embedding dimension 2:", EMBED_DIM_2)

Embedding dimension 1: 3072
Embedding dimension 2: 3072


In [ ]:
vec = embeddings.embed_query("another random query")
EMBED_DIM = len(vec)

print("Embedding dimension:", EMBED_DIM)


### Pinecone vector db index

In [82]:
import os
from pinecone import Pinecone, ServerlessSpec

# os.environ["PINECONE_API_KEY"] = "..."

PINECONE_INDEX_NAME = "contractiq-v1"
PINECONE_CLOUD = "aws"      
PINECONE_REGION = "us-east-1"

pc = Pinecone(api_key=PINECONE_API_KEY)  # :contentReference[oaicite:5]{index=5}

pc.list_indexes()

[
    {
        "name": "stoicmind",
        "metric": "cosine",
        "host": "stoicmind-vnshmuq.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "region": "us-east-1",
                "cloud": "aws",
                "read_capacity": {
                    "mode": "OnDemand",
                    "status": {
                        "state": "Ready",
                        "current_shards": null,
                        "current_replicas": null
                    }
                }
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 384,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "contractiq-v1",
        "metric": "cosine",
        "host": "contractiq-v1-vnshmuq.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "region": "us-east-1",

In [84]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import os

local_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
LOCAL_DIM = local_model.get_sentence_embedding_dimension()
print("Local embedding dim:", LOCAL_DIM)  # usually 384


pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

LOCAL_INDEX_NAME = "contractiq-local-v1"
existing = [ix["name"] for ix in pc.list_indexes()]
if LOCAL_INDEX_NAME not in existing:
    pc.create_index(
        name=LOCAL_INDEX_NAME,
        dimension=LOCAL_DIM,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

local_index = pc.Index(LOCAL_INDEX_NAME)
print("Local Pinecone index ready:", LOCAL_INDEX_NAME)


Local embedding dim: 384
Local Pinecone index ready: contractiq-local-v1


In [85]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT rowid FROM documents"))
    # Get column names
    column_names = result.keys()
    print("Column names:", list(column_names))
    # Fetch the data
    documents = result.fetchmany(5)
    
for r in documents:
    
    print(r)

Column names: ['rowid']
(5,)
(3,)
(4,)
(1,)
(6,)


In [86]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM documents"))
    # Get column names
    column_names = result.keys()
    print("Column names:", list(column_names))
    # Fetch the data
    documents = result.fetchmany(5)
    
for r in documents:
    
    print(r)

Column names: ['doc_id', 'title', 'source', 'text']
('b26c38227ce11a4e', 'NeuroboPharmaceuticalsInc_20190903_S-4_EX-10.36_11802165_EX-10.36_Manufacturing Agreement_ Supply Agreement', 'cuad-qa', 'Exhibit 10.36 [Pursuant to Item 601(b)(10) of Regulation S-K, certain confidential portions of this exhibit have been omitted by means of marking suc ... (19798 characters truncated) ...  PHARMACEUTICALS, INC., S-4, 9/3/2019\n\n\n\n\n\nSchedule 4.2 Supply Price [***] Schedule 1-5\n\nSource: NEUROBO PHARMACEUTICALS, INC., S-4, 9/3/2019')
('f71b4149a185d016', 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT', 'cuad-qa', 'EXHIBIT 10.6\n\n                              DISTRIBUTOR AGREEMENT\n\n         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and betwe ... (54332 characters truncated) ... tump       -------------------                       -------------------------------       President\n\n                                    Page -14-')
('43ab152a17a15599', 'WHITESMOKE,INC_1

In [47]:
def fetch_chunk_batch(conn, last_rowid: int, batch_size: int):
    sql = text("""
        SELECT
            c.rowid AS rowid,
            c.chunk_id,
            c.doc_id,
            c.chunk_index,
            c.start_char,
            c.end_char,
            c.text AS chunk_text,
            d.title AS doc_title,
            d.source AS doc_source
        FROM chunks c
        JOIN documents d ON d.doc_id = c.doc_id
        WHERE c.rowid > :last_rowid
        ORDER BY c.rowid
        LIMIT :batch_size
    """)
    rows = conn.execute(sql, {"last_rowid": last_rowid, "batch_size": batch_size})
    column_names = list(rows.keys())
    row_names_dict = {i: col_name for i, col_name in enumerate(column_names)}
    print('clumn names:', row_names_dict)
    rows = rows.fetchall()
    return rows


In [48]:
with engine.connect() as conn:
    sql_batch = fetch_chunk_batch(conn, 0, 5)

for r in sql_batch:
    print(r)

clumn names: {0: 'rowid', 1: 'chunk_id', 2: 'doc_id', 3: 'chunk_index', 4: 'start_char', 5: 'end_char', 6: 'chunk_text', 7: 'doc_title', 8: 'doc_source'}
(1, 'ff7f3a9d475478f8', 'b26c38227ce11a4e', 0, 0, 1000, 'Exhibit 10.36 [Pursuant to Item 601(b)(10) of Regulation S-K, certain confidential portions of this exhibit have been omitted by means of marking suc ... (710 characters truncated) ... t 64 Cheonho-daero, Dongdaemun-gu, Seoul 02587, Republic of Korea ("Dong-A") and NeuroBo Pharmaceuticals, Inc., a corporation duly incorporated under', 'NeuroboPharmaceuticalsInc_20190903_S-4_EX-10.36_11802165_EX-10.36_Manufacturing Agreement_ Supply Agreement', 'cuad-qa')
(2, 'c7061889511944bd', 'b26c38227ce11a4e', 1, 850, 1850, 'at 64 Cheonho-daero, Dongdaemun-gu, Seoul 02587, Republic of Korea ("Dong-A") and NeuroBo Pharmaceuticals, Inc., a corporation duly incorporated unde ... (702 characters truncated) ... ducts and their matching placebo for the purpose of research and development of the Li

### Data stored in pinecone

for each chunk, pinecone will store one record like:
- id (chunk id)
- vector: embedding of `chunk_text`
- metadata (`doc_id, chunk_index, start_char, end_char, title, source, preview`)

### gemini embedding has batch limits, we use `EMBED_BATCH_SIZE=100` to be safe

In [79]:
def embed_documents_with_retry(texts, batch_size: int, max_retries: int = 30):
    tries = 0
    while True:
        try:
            # Force internal batching to avoid many small requests
            return embeddings.embed_documents(texts, batch_size=batch_size)
        except Exception as e:
            msg = str(e)
            if ("429" not in msg) and ("RESOURCE_EXHAUSTED" not in msg):
                raise

            delay = 10.0
            m = re.search(r"Please retry in\s+([0-9.]+)s", msg)
            if m:
                delay = float(m.group(1)) + 0.5

            tries += 1
            if tries > max_retries:
                raise

            print(f"[429] sleeping {delay:.1f}s then retrying (attempt {tries}/{max_retries})")
            time.sleep(delay)


In [80]:
import time

def normalize_vector(v):
    # v may already be a list[float], or a list-like
    return [float(x) for x in list(v)]

def upsert_chunk_rows(rows, namespace: str, embed_batch_size: int = 100):
    # 1) prepare texts for embeddings
    texts = [r[6] for r in rows]  # chunk_text

    # 2) embed documents (batch)
    # vecs = embeddings.embed_documents(texts, batch_size=100)  # :contentReference[oaicite:1]{index=1}
    vecs = embed_documents_with_retry(texts, batch_size=embed_batch_size)


    # 3) build Pinecone records
    vectors = []
    for r, v in zip(rows, vecs):
        rowid = int(r[0])
        chunk_id = r[1]
        doc_id = r[2]
        chunk_index = int(r[3])
        start_char = int(r[4])
        end_char = int(r[5])
        chunk_text = r[6]
        doc_title = r[7]
        doc_source = r[8]

        meta = {
            "doc_id": doc_id,
            "chunk_index": chunk_index,
            "start_char": start_char,
            "end_char": end_char,
            "title": doc_title,
            "source": doc_source,
            # keep metadata small; store only a short preview (optional)
            "preview": chunk_text[:200],
        }

        vectors.append({
            "id": chunk_id,
            "values": normalize_vector(v),
            "metadata": meta,
        })

    # 4) upsert to Pinecone
    index.upsert(vectors=vectors, namespace=namespace)  # :contentReference[oaicite:2]{index=2}


### Run full indexing loop

In [69]:
NAMESPACE = "cuad-chunks-v1"

SQL_BATCH_SIZE = 200       # how many rows to fetch from SQLite at a time
EMBED_BATCH_SIZE = 100      # how many chunks to embed/upsert at a time (safe default)

# quick “how many chunks do we expect?”
with engine.connect() as conn:
    total_chunks = conn.execute(text("SELECT COUNT(*) FROM chunks")).scalar_one()
print("Total chunks in SQLite:", total_chunks)

Total chunks in SQLite: 224


### Check if any embedding calls can run right now or if we hit the gemini limits

In [76]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

emb = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
try:
    v = emb.embed_query("ping")
    print("OK, dim =", len(v))
except Exception as e:
    print("FAILED:", e)


OK, dim = 3072


In [77]:
texts = ["ping one", "ping two", "ping three", "ping four", "ping five"]
vecs = embeddings.embed_documents(texts, batch_size=100)
print("embedded:", len(vecs), "dim:", len(vecs[0]))


embedded: 5 dim: 3072


In [81]:
NAMESPACE = "cuad-chunks-v1"
SQL_BATCH_SIZE = 100  # <= 100 so upsert_chunk_rows makes one embed call
last_rowid = 0
upserted = 0

with engine.connect() as conn:
    total_chunks = conn.execute(text("SELECT COUNT(*) FROM chunks")).scalar_one()
print("Total chunks in SQLite:", total_chunks)

with engine.connect() as conn:
    while True:
        rows = fetch_chunk_batch(conn, last_rowid=last_rowid, batch_size=SQL_BATCH_SIZE)
        if not rows:
            break

        last_rowid = int(rows[-1][0])

        upsert_chunk_rows(rows, namespace=NAMESPACE, embed_batch_size=100)
        upserted += len(rows)
        print(f"upserted {upserted}/{total_chunks}")

        time.sleep(0.5)  # optional


Total chunks in SQLite: 224
clumn names: {0: 'rowid', 1: 'chunk_id', 2: 'doc_id', 3: 'chunk_index', 4: 'start_char', 5: 'end_char', 6: 'chunk_text', 7: 'doc_title', 8: 'doc_source'}
[429] sleeping 16.1s then retrying (attempt 1/30)
[429] sleeping 59.7s then retrying (attempt 2/30)
[429] sleeping 59.6s then retrying (attempt 3/30)
[429] sleeping 59.7s then retrying (attempt 4/30)
[429] sleeping 59.7s then retrying (attempt 5/30)
[429] sleeping 59.7s then retrying (attempt 6/30)
[429] sleeping 59.7s then retrying (attempt 7/30)


KeyboardInterrupt: 

### Upsert using local embedding model

### local embedding using sentence-transformers/all-MiniLM-L6-v2

In [91]:
from sentence_transformers import SentenceTransformer
local_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cpu")
LOCAL_DIM = local_model.get_sentence_embedding_dimension()
print("Local embedding dim:", LOCAL_DIM)  # usually 384

Local embedding dim: 384


In [92]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

LOCAL_INDEX_NAME = "contractiq-local-v1"
existing = [ix["name"] for ix in pc.list_indexes()]
if LOCAL_INDEX_NAME not in existing:
    pc.create_index(
        name=LOCAL_INDEX_NAME,
        dimension=LOCAL_DIM,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

local_index = pc.Index(LOCAL_INDEX_NAME)
print("Local Pinecone index ready:", LOCAL_INDEX_NAME)

Local Pinecone index ready: contractiq-local-v1


In [93]:
LOCAL_NAMESPACE = "cuad-chunks-local-v1"
SQL_BATCH_SIZE = 200
EMBED_BATCH_SIZE = 64  # local batch size for speed

def upsert_chunk_rows_local(rows, namespace: str):
    texts = [r[6] for r in rows]  # chunk_text

    # local embeddings (no API calls)
    vecs = local_model.encode(
        texts,
        batch_size=EMBED_BATCH_SIZE,
        show_progress_bar=False,
        normalize_embeddings=True,   # helpful for cosine
    )

    vectors = []
    for r, v in zip(rows, vecs):
        vectors.append({
            "id": r[1],  # chunk_id
            "values": [float(x) for x in v],
            "metadata": {
                "doc_id": r[2],
                "chunk_index": int(r[3]),
                "start_char": int(r[4]),
                "end_char": int(r[5]),
                "title": r[7],
                "source": r[8],
                "preview": r[6][:200],
            }
        })

    local_index.upsert(vectors=vectors, namespace=namespace)

In [94]:
# run full loop
with engine.connect() as conn:
    total_chunks = conn.execute(text("SELECT COUNT(*) FROM chunks")).scalar_one()
print("Total chunks:", total_chunks)

last_rowid = 0
upserted = 0

with engine.connect() as conn:
    while True:
        rows = fetch_chunk_batch(conn, last_rowid=last_rowid, batch_size=SQL_BATCH_SIZE)
        if not rows:
            break
        last_rowid = int(rows[-1][0])

        upsert_chunk_rows_local(rows, namespace=LOCAL_NAMESPACE)
        upserted += len(rows)
        print(f"upserted {upserted}/{total_chunks}")

print("DONE")

Total chunks: 224
clumn names: {0: 'rowid', 1: 'chunk_id', 2: 'doc_id', 3: 'chunk_index', 4: 'start_char', 5: 'end_char', 6: 'chunk_text', 7: 'doc_title', 8: 'doc_source'}
upserted 200/224
clumn names: {0: 'rowid', 1: 'chunk_id', 2: 'doc_id', 3: 'chunk_index', 4: 'start_char', 5: 'end_char', 6: 'chunk_text', 7: 'doc_title', 8: 'doc_source'}
upserted 224/224
clumn names: {0: 'rowid', 1: 'chunk_id', 2: 'doc_id', 3: 'chunk_index', 4: 'start_char', 5: 'end_char', 6: 'chunk_text', 7: 'doc_title', 8: 'doc_source'}
DONE


In [110]:
local_index.describe_index_stats()

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '194',
                                    'content-type': 'application/json',
                                    'date': 'Mon, 05 Jan 2026 20:07:36 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '93',
                                    'x-pinecone-request-id': '5333589105358587057',
                                    'x-pinecone-request-latency-ms': '89',
                                    'x-pinecone-response-duration-ms': '99'}},
 'dimension': 384,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'cuad-chunks-local-v1': {'vector_count': 224}},
 'storageFullness': 0.0,
 'total_vector_count': 224,
 'vector_type': 'dense'}

### Query pinecone and pull matching chunk text from SQLite

In [96]:
import numpy as np

QUESTION = "What is the effective date of this agreement?"

# 1) Embed the question with the SAME model you used for indexing
q_vec = local_model.encode([QUESTION], normalize_embeddings=True)
print(q_vec)
print(q_vec.shape)

[[ 1.42644374e-02  4.39775065e-02  5.39595038e-02  1.07883783e-02
  -7.53192380e-02  4.04509380e-02 -5.05269766e-02  4.19914257e-04
  -2.82138214e-02  2.40832381e-02  3.26010101e-02  8.11804309e-02
  -3.91404442e-02  6.09899638e-03 -8.34050775e-03  6.12314045e-02
   1.66095644e-02 -5.09771816e-02  3.09583079e-03  4.31748219e-02
  -2.68656798e-02 -1.57065578e-02 -1.52412169e-02  1.62910484e-02
   3.53858881e-02 -2.19053607e-02  3.38817984e-02 -3.43579464e-02
   4.12362218e-02  3.84547794e-03 -4.32771444e-03  4.13448364e-02
   6.18377887e-02  2.49186736e-02 -5.91558032e-03 -4.62551117e-02
   2.09093057e-02 -6.17092550e-02  4.49442305e-02  8.37223511e-03
  -5.59657998e-02 -9.18704495e-02 -1.71847118e-03 -3.60682756e-02
  -4.79508042e-02  3.53880338e-02  2.41745170e-02  1.03433035e-01
  -6.43571913e-02  8.16837922e-02  7.38783460e-03 -2.99672037e-03
   1.70370063e-03  2.03112559e-03 -3.60072614e-03  2.63953861e-02
  -8.51479098e-02 -8.48796126e-03  2.83556357e-02  2.64556147e-02
   6.99275

In [97]:
q_vec = local_model.encode([QUESTION], normalize_embeddings=True)[0]
q_vec = [float(x) for x in q_vec]  # Pinecone wants plain Python floats

# 2) Query Pinecone
TOP_K = 5
res = local_index.query(
    vector=q_vec,
    top_k=TOP_K,
    namespace=LOCAL_NAMESPACE,
    include_metadata=True
)
print(res)

QueryResponse(matches=[{'id': '037496b638d3efcf',
 'metadata': {'chunk_index': 6,
              'doc_id': 'f71b4149a185d016',
              'end_char': 6100,
              'preview': '                       Agreement,  may be executed  '
                         'for each state or entity                            '
                         'representing each state.\n'
                         '\n'
                         '\n'
                         '\n'
                         '\n'
                         '\n'
                         '                  1.3      Term.  The term of this  '
                         'Agreement  sha',
              'source': 'cuad-qa',
              'start_char': 5100,
              'title': 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT'},
 'score': 0.577136099,
 'values': []}, {'id': '7f34ebd71de25b21',
 'metadata': {'chunk_index': 37,
              'doc_id': 'f71b4149a185d016',
              'end_char': 32450,
              'preview': ' (90)

In [100]:
print(dir(res))

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_response_info', 'get', 'matches', 'namespace', 'usage']


In [102]:

# 3) Print results
matches = res.get("matches", [])
print("matches:", len(matches))
for m in matches:
    print("score:", round(m["score"], 4), "| id:", m["id"], "| doc_id:", m["metadata"].get("doc_id"))


matches: 5
score: 0.5771 | id: 037496b638d3efcf | doc_id: f71b4149a185d016
score: 0.5341 | id: 7f34ebd71de25b21 | doc_id: f71b4149a185d016
score: 0.501 | id: 53ce1548c93503ad | doc_id: b26c38227ce11a4e
score: 0.4987 | id: 23ea8a4926c01282 | doc_id: f71b4149a185d016
score: 0.4913 | id: 9c4b99493d517d89 | doc_id: 43ab152a17a15599


In [103]:
match_chunk_ids = [m["id"] for m in matches]
print(match_chunk_ids)

['037496b638d3efcf', '7f34ebd71de25b21', '53ce1548c93503ad', '23ea8a4926c01282', '9c4b99493d517d89']


In [106]:
params={}
placeholders=[]
for i, c in enumerate(match_chunk_ids):
    key = f"id{i}"
    placeholders.append(f":{key}")
    params[key] = c
print(params)
print(placeholders)

{'id0': '037496b638d3efcf', 'id1': '7f34ebd71de25b21', 'id2': '53ce1548c93503ad', 'id3': '23ea8a4926c01282', 'id4': '9c4b99493d517d89'}
[':id0', ':id1', ':id2', ':id3', ':id4']


In [107]:
print(", ".join(placeholders))

:id0, :id1, :id2, :id3, :id4


### Fetch those chunks from SQLite by chunk_id

In [108]:
def fetch_chunks_by_ids(conn, chunk_ids):
    if len(chunk_ids) == 0:
        return []

    # build placeholders (:id0, :id1, ...)
    params = {}
    placeholders = []
    for i, cid in enumerate(chunk_ids):
        key = f"id{i}"
        placeholders.append(f":{key}")
        params[key] = cid

    sql = f"""
        SELECT chunk_id, doc_id, chunk_index, start_char, end_char, text
        FROM chunks
        WHERE chunk_id IN ({", ".join(placeholders)})
    """

    rows = conn.execute(text(sql), params).fetchall()

    # Keep the order same as chunk_ids
    by_id = {r[0]: r for r in rows}
    ordered = []
    for cid in chunk_ids:
        if cid in by_id:
            r = by_id[cid]
            ordered.append({
                "chunk_id": r[0],
                "doc_id": r[1],
                "chunk_index": int(r[2]),
                "start_char": int(r[3]),
                "end_char": int(r[4]),
                "text": r[5],
            })
    return ordered


### pull and preview retrieved chunks

In [109]:
chunk_ids = [m["id"] for m in matches]

with engine.connect() as conn:
    retrieved_chunks = fetch_chunks_by_ids(conn, chunk_ids)

print("retrieved_chunks:", len(retrieved_chunks))
for c in retrieved_chunks:
    print("\n---", c["chunk_id"], "| doc:", c["doc_id"], "| idx:", c["chunk_index"])
    print(c["text"][:400])


retrieved_chunks: 5

--- 037496b638d3efcf | doc: f71b4149a185d016 | idx: 6
                       Agreement,  may be executed  for each state or entity                            representing each state.





                  1.3      Term.  The term of this  Agreement  shall be ten (10)                            years (the "Term")  which shall  commence on the date                            upon which the Company  delivers to  Distributor  the                        

--- 7f34ebd71de25b21 | doc: f71b4149a185d016 | idx: 37
 (90) days after  execution of this  Agreement,  describing its                   efforts,  the potential  customers it has  approached  and the                   status of its efforts.

         4.2      Termination  for  Cause.   Either  party  may  terminate  this                   Agreement upon 30 days

                                    Page -8-

                  prior written  notice to t

--- 53ce1548c93503ad | doc: b26c38227ce11a4e | idx: 22
e as it is 

### gemini answer with chunk-id citation

In [111]:
import json
import re
from sqlalchemy import text
from langchain_google_genai import ChatGoogleGenerativeAI

# 1) LLM (generation only)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",  # model code :contentReference[oaicite:2]{index=2}
    temperature=0.0,
)
print(llm)

profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True} google_api_key=SecretStr('**********') model='gemini-2.5-flash-lite' temperature=0.0 client=<google.genai.client.Client object at 0x74888be58690> default_metadata=() model_kwargs={}


In [117]:
def build_context(chunks, max_chars_per_chunk: int = 1200) -> str:
    lines = []
    for c in chunks:
        header = (
            f"[{c['chunk_id']}] doc_id={c['doc_id']} "
            f"chunk_index={c['chunk_index']} "
            f"chars={c['start_char']}-{c['end_char']}"
        )
        body = c["text"][:max_chars_per_chunk]
        lines.append(header + "\n" + body)
    return "\n\n---\n\n".join(lines)

In [113]:
import re
def parse_json_from_text(s: str) -> dict:
    """Try to parse strict JSON; if model wraps it, extract the first {...} block."""
    s = s.strip()
    try:
        return json.loads(s)
    except Exception:
        m = re.search(r"\{.*\}", s, flags=re.DOTALL)
        if not m:
            raise
        return json.loads(m.group(0))


In [115]:
system = (
        "You are a careful contract assistant.\n"
        "Use ONLY the provided EXCERPTS to answer.\n"
        "If the answer is not clearly present, say you can't find it.\n"
        "Return STRICT JSON (no markdown) with keys:\n"
        "  answer: string\n"
        "  citations: array of chunk_id strings you used\n"
        "Citations MUST be chosen from the bracketed ids in the excerpts."
    )
ai_msg = llm.invoke([("system", system), ("human", QUESTION)])
print(ai_msg)

content='```json\n{\n "answer": "I cannot find the effective date of this agreement in the provided text.",\n "citations": []\n}\n```' additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019b8fd0-68c3-71c2-9f78-82b1ce559ed4-0' usage_metadata={'input_tokens': 88, 'output_tokens': 32, 'total_tokens': 120, 'input_token_details': {'cache_read': 0}}


In [116]:
ai_msg.content

'```json\n{\n "answer": "I cannot find the effective date of this agreement in the provided text.",\n "citations": []\n}\n```'

In [120]:
def extract_ai_text(ai_msg) -> str:
    """LangChain sometimes returns content as a string or as a list of parts."""
    c = ai_msg.content
    if isinstance(c, str):
        return c
    if isinstance(c, list):
        # common shape: [{'type':'text','text':'...'}, ...]
        out = []
        for part in c:
            if isinstance(part, dict) and "text" in part:
                out.append(part["text"])
        return "\n".join(out).strip()
    return str(c)


In [146]:
def answer_with_citations(
    question: str,
    top_k: int = 7,
    *,
    doc_id: str | None = None,
    namespace: str = "cuad-chunks-local-v1",
    max_chars_per_chunk: int = 1200,
):
    """
    End-to-end RAG with evidence quotes + guardrails.
    * means everyting after * must be passed by named keyword

    This Assumes these are already defined in your notebook:
      - local_model (SentenceTransformer)
      - local_index (Pinecone Index)
      - engine (SQLAlchemy engine)
      - llm (ChatGoogleGenerativeAI)
      - fetch_chunks_by_ids(conn, chunk_ids) -> list[dict]
      - verify_citations(result_obj) -> bool

    Also assumes these helper functions exist (no nesting):
      - build_context(chunks, max_chars_per_chunk) -> str
      - extract_ai_text(ai_msg) -> str
      - parse_json_from_text(s: str) -> dict
    """

    # A) Retrieve chunk IDs from Pinecone (local embeddings)
    q_vec = local_model.encode([question], normalize_embeddings=True)[0]
    q_vec = [float(x) for x in q_vec]

    # res = local_index.query(
    #     vector=q_vec,
    #     top_k=top_k,
    #     namespace=namespace,
    #     include_metadata=True,
    # )
    query_kwargs = {
    "vector": q_vec,
    "top_k": top_k,
    "namespace": namespace,
    "include_metadata": True,
    }

    # If doc_id is set, only retrieve chunks from that contract
    if doc_id is not None:
        query_kwargs["filter"] = {"doc_id": {"$eq": doc_id}}

    res = local_index.query(**query_kwargs)

    matches = res.get("matches", [])
    chunk_ids = [m["id"] for m in matches]

    # B) Fetch full chunk texts from SQLite
    with engine.connect() as conn:
        chunks = fetch_chunks_by_ids(conn, chunk_ids)

    # C) Build context and ask Gemini for strict JSON with quotes
    context = build_context(chunks, max_chars_per_chunk=max_chars_per_chunk)

    system = (
        "You are a careful contract assistant.\n"
        "Use ONLY the provided EXCERPTS to answer.\n"
        "If the answer is not clearly present, say you can't find it.\n"
        "Return STRICT JSON (no markdown) with keys:\n"
        "  answer: string\n"
        "  citations: array of objects with keys {chunk_id: string, quote: string}\n"
        "Rules:\n"
        "- chunk_id must be one of the bracketed ids in the excerpts.\n"
        "- quote must be copied verbatim from that chunk and directly support the answer.\n"
    )

    prompt = (
        f"QUESTION:\n{question}\n\n"
        f"EXCERPTS:\n{context}\n\n"
        "Return JSON now."
    )

    ai_msg = llm.invoke([("system", system), ("human", prompt)])
    raw = extract_ai_text(ai_msg)
    data = parse_json_from_text(raw)

    # D) Normalize citations to a strict list of {chunk_id, quote}
    citations = data.get("citations", [])
    if citations is None:
        citations = []

    clean = []
    for item in citations:
        if isinstance(item, dict) and "chunk_id" in item and "quote" in item:
            clean.append({"chunk_id": item["chunk_id"], "quote": item["quote"]})

    result_obj = {
        "question": question,
        "answer": data.get("answer"),
        "citations": clean,
        "retrieved_chunk_ids": chunk_ids,
        "raw_model_output": raw,
    }

    # E) Guardrails
    if len(result_obj["citations"]) == 0:
        result_obj["answer"] = "I can't find this answer in the provided excerpts."
        return result_obj

    if not verify_citations(result_obj):
        result_obj["answer"] = "I can't verify the evidence in the provided excerpts."
        result_obj["citations"] = []
        return result_obj

    return result_obj


In [147]:
result = answer_with_citations("What is the effective date of this agreement?", top_k=5)
print(result)

[BAD] quote not found verbatim in chunk 037496b638d3efcf
quote: The term of this Agreement shall be ten (10) years (the "Term") which shall commence on the date upon which the Company delivers to Distributor the last Sample, as defined hereinafter.
{'question': 'What is the effective date of this agreement?', 'answer': "I can't verify the evidence in the provided excerpts.", 'citations': [], 'retrieved_chunk_ids': ['037496b638d3efcf', '7f34ebd71de25b21', '53ce1548c93503ad', '23ea8a4926c01282', '9c4b99493d517d89'], 'raw_model_output': '```json\n{\n "answer": "The effective date of this agreement is the date on which the Company delivers the last Sample to the Distributor. This marks the commencement of the agreement\'s ten-year term.",\n "citations": [\n  {\n   "chunk_id": "037496b638d3efcf",\n   "quote": "The term of this Agreement shall be ten (10) years (the \\"Term\\") which shall commence on the date upon which the Company delivers to Distributor the last Sample, as defined hereina

In [123]:
print("ANSWER:", result["answer"])
print("CITATIONS:", result["citations"])
print("RETRIEVED:", result["retrieved_chunk_ids"])


ANSWER: The effective date of this agreement is the date it is signed by both parties. The agreement specifies that it is "as of the Effective Date" when the parties have executed it. However, a specific calendar date for the effective date is not provided in the excerpts.
CITATIONS: ['53ce1548c93503ad']
RETRIEVED: ['037496b638d3efcf', '7f34ebd71de25b21', '53ce1548c93503ad', '23ea8a4926c01282', '9c4b99493d517d89']


In [124]:
def print_chunks_by_ids(chunk_ids):
    with engine.connect() as conn:
        chunks_list = fetch_chunks_by_ids(conn, chunk_ids)

    print("chunks found:", len(chunks_list))
    for c in chunks_list:
        print("\n==============================")
        print("chunk_id:", c["chunk_id"])
        print("doc_id:", c["doc_id"], "chunk_index:", c["chunk_index"])
        print("char range:", c["start_char"], "-", c["end_char"])
        print("------------------------------")
        print(c["text"])

print_chunks_by_ids(result["citations"])


chunks found: 1

chunk_id: 53ce1548c93503ad
doc_id: b26c38227ce11a4e chunk_index: 22
char range: 18700 - 19700
------------------------------
e as it is set forth in this Agreement <Signature page follows.> 6

Source: NEUROBO PHARMACEUTICALS, INC., S-4, 9/3/2019





IN WITNESS WHEREOF, the Parties hereto have duly executed this Agreement as of the Effective Date. DONG-A ST CO., LTD. By: /s/ Daesik Eom Daesik Eom, Chairman and CEO NEUROBO PHARMACEUTICALS, INC. By: /s/ John L. Brooks, III John L. Brooks III, President & CEO 7

Exhibit A Product Specifications

Test Analytical Method Specification Requirements [***] [***] [***]

[***] [***] [***][***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] Schedule 1-1

Source: NEUROBO PHARMACEUTICALS, INC., S-4, 9/3/2019





Exhibit B Product Test Methods [***] Schedule 1-2

Source: NEUROBO PHARMACEUTICALS, INC., S-4, 9/3/2019





Exhibit C API Specifications

Test Analytical M

In [125]:
print_chunks_by_ids(result["citations"])

chunks found: 1

chunk_id: 53ce1548c93503ad
doc_id: b26c38227ce11a4e chunk_index: 22
char range: 18700 - 19700
------------------------------
e as it is set forth in this Agreement <Signature page follows.> 6

Source: NEUROBO PHARMACEUTICALS, INC., S-4, 9/3/2019





IN WITNESS WHEREOF, the Parties hereto have duly executed this Agreement as of the Effective Date. DONG-A ST CO., LTD. By: /s/ Daesik Eom Daesik Eom, Chairman and CEO NEUROBO PHARMACEUTICALS, INC. By: /s/ John L. Brooks, III John L. Brooks III, President & CEO 7

Exhibit A Product Specifications

Test Analytical Method Specification Requirements [***] [***] [***]

[***] [***] [***][***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [***] Schedule 1-1

Source: NEUROBO PHARMACEUTICALS, INC., S-4, 9/3/2019





Exhibit B Product Test Methods [***] Schedule 1-2

Source: NEUROBO PHARMACEUTICALS, INC., S-4, 9/3/2019





Exhibit C API Specifications

Test Analytical M

In [126]:
print_chunks_by_ids(result["retrieved_chunk_ids"])

chunks found: 5

chunk_id: 037496b638d3efcf
doc_id: f71b4149a185d016 chunk_index: 6
char range: 5100 - 6100
------------------------------
                       Agreement,  may be executed  for each state or entity                            representing each state.





                  1.3      Term.  The term of this  Agreement  shall be ten (10)                            years (the "Term")  which shall  commence on the date                            upon which the Company  delivers to  Distributor  the                            last Sample, as defined  hereinafter.  If Distributor                            complies with all of the terms of this Agreement, the                            Agreement  shall be  renewable on an annual basis for                            one (1) year terms for up to  another  ten (10) years                            on the same terms and conditions as set forth herein.                            All renewals of this  Agreement  shall be on the sam

### the date is not shown, is it in a neighboring chunk?

In [127]:
from sqlalchemy import text

cited_chunk_id = result["citations"][0]

with engine.connect() as conn:
    row = conn.execute(text("""
        SELECT doc_id, chunk_index
        FROM chunks
        WHERE chunk_id = :cid
    """), {"cid": cited_chunk_id}).fetchone()

doc_id = row[0]
chunk_index = int(row[1])

print("cited_chunk_id:", cited_chunk_id)
print("doc_id:", doc_id, "chunk_index:", chunk_index)

with engine.connect() as conn:
    neigh = conn.execute(text("""
        SELECT chunk_id, chunk_index, text
        FROM chunks
        WHERE doc_id = :doc_id AND chunk_index IN (:a, :b, :c)
        ORDER BY chunk_index
    """), {"doc_id": doc_id, "a": chunk_index - 1, "b": chunk_index, "c": chunk_index + 1}).fetchall()

for r in neigh:
    print("\n==============================")
    print("chunk_index:", int(r[1]), "chunk_id:", r[0])
    print(r[2][-400:])  # last 400 chars
    print("\n--- NEXT CHUNK START ---\n")
    print(r[2][:400])   # first 400 chars


cited_chunk_id: 53ce1548c93503ad
doc_id: b26c38227ce11a4e chunk_index: 22

chunk_index: 21 chunk_id: 025f72c8f69389fc
ssignment), 12.4 (Severability), 12.6 (Remedies), 12.8 (Submission to Jurisdiction/Waiver of Jury Trial), 12.9 (Independent Contractor/No Agency), 12.10 (Entire Agreement) of the License Agreement shall be deemed to be incorporated herein by reference as it is set forth in this Agreement <Signature page follows.> 6

Source: NEUROBO PHARMACEUTICALS, INC., S-4, 9/3/2019





IN WITNESS WHEREOF, the 

--- NEXT CHUNK START ---

ivered in writing by courier, electronic facsimile (fax), or certified or registered mail, (postage prepaid and return receipt requested) to the other Party; and shall be effective upon receipt or three (3) business days after being deposited in the mail, whichever occurs sooner. Notices to the Parties shall be sent to the addresses set forth at the beginning of this Agreement. Notice of change of

chunk_index: 22 chunk_id: 53ce1548c93503ad
[***] [**

In [128]:
with engine.connect() as conn:
    neigh = conn.execute(text("""
        SELECT chunk_id, chunk_index, text
        FROM chunks
        WHERE doc_id = :doc_id AND chunk_index BETWEEN :lo AND :hi
        ORDER BY chunk_index
    """), {"doc_id": doc_id, "lo": chunk_index - 1, "hi": chunk_index + 1}).fetchall()

for r in neigh:
    print("\n==============================")
    print("chunk_index:", int(r[1]), "chunk_id:", r[0])
    print("TAIL:\n", r[2][-300:])
    print("\nHEAD:\n", r[2][:300])



chunk_index: 21 chunk_id: 025f72c8f69389fc
TAIL:
 ial), 12.9 (Independent Contractor/No Agency), 12.10 (Entire Agreement) of the License Agreement shall be deemed to be incorporated herein by reference as it is set forth in this Agreement <Signature page follows.> 6

Source: NEUROBO PHARMACEUTICALS, INC., S-4, 9/3/2019





IN WITNESS WHEREOF, the 

HEAD:
 ivered in writing by courier, electronic facsimile (fax), or certified or registered mail, (postage prepaid and return receipt requested) to the other Party; and shall be effective upon receipt or three (3) business days after being deposited in the mail, whichever occurs sooner. Notices to the Part

chunk_index: 22 chunk_id: 53ce1548c93503ad
TAIL:
 

Exhibit B Product Test Methods [***] Schedule 1-2

Source: NEUROBO PHARMACEUTICALS, INC., S-4, 9/3/2019





Exhibit C API Specifications

Test Analytical Method Specification Requirements [***] [***] [***]

[***] [***] [***][***] [***] [***] [***] [***] [***] [***] [***] [***] [***] [

In [140]:
q2 = "What is the governing law of this agreement?"
result2 = answer_with_citations(q2, top_k=7)

print("Q:", result2["question"])
print("ANSWER:", result2["answer"])
print("CITATIONS:", result2["citations"])
print("RETRIEVED:", result2["retrieved_chunk_ids"])


Q: What is the governing law of this agreement?
ANSWER: This Agreement is governed by English law, and any disputes related to it will be subject to the exclusive jurisdiction of the English courts.
CITATIONS: [{'chunk_id': '7e729bc4b31d5c25', 'quote': '11.10 Governing Law. This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an injunction or other relief to protect its Intellectual Property Rights.'}]
RETRIEVED: ['7e729bc4b31d5c25', 'e3d38988b60749fa', '037496b638d3efcf', '2e84d90c77bb4757', '7590ad6f4629b3a0', 'b089bfe1d37de2c1', '9dff04734fc89156']


In [130]:
print_chunks_by_ids(result2["citations"])

chunks found: 1

chunk_id: 7e729bc4b31d5c25
doc_id: 43ab152a17a15599 chunk_index: 48
char range: 40800 - 41800
------------------------------
expressly stated otherwise, nothing in this Agreement shall create an agency, partnership or joint venture of any  kind between the parties.

11.9 No Third-Party Beneficiaries. Except as expressly stated otherwise, nothing in this Agreement shall create or confer any rights or other  benefits in favour of any person other than the parties to this Agreement.

  - 13 -











   Signed by the parties on the dates shown below.

   *Confidential treatment requested  Google Confidential



11.10 Governing Law. This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights. If this Agree

In [133]:
result3 = answer_with_citations("What is the governing law of this agreement?", top_k=7)
print("ANSWER:", result3["answer"])
print("CITATIONS:", result3["citations"])


ANSWER: This agreement is governed by English law, and any disputes related to it (whether contractual or non-contractual) will be subject to the exclusive jurisdiction of the English courts. However, either party can apply to any court for an injunction or other relief to protect their Intellectual Property Rights. In case of any conflict, the English text of the agreement will take precedence.
CITATIONS: [{'chunk_id': '7e729bc4b31d5c25', 'quote': '11.10 Governing Law. This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights. If this Agreement is translated into any other language, if there is  conflict the English text will take precedence.'}]


In [136]:
def verify_citations(result_obj):
    """
    Checks:
      - each cited chunk_id exists
      - the provided quote appears verbatim inside that chunk text
    """
    citations = result_obj["citations"]
    ids = [c["chunk_id"] for c in citations]

    with engine.connect() as conn:
        chunks_list = fetch_chunks_by_ids(conn, ids)

    by_id = {c["chunk_id"]: c["text"] for c in chunks_list}

    ok = True
    for c in citations:
        cid = c["chunk_id"]
        quote = c["quote"]

        if cid not in by_id:
            ok = False
            print(f"[BAD] chunk_id not found in SQLite: {cid}")
            continue

        if quote not in by_id[cid]:
            ok = False
            print(f"[BAD] quote not found verbatim in chunk {cid}")
            print("quote:", quote)
        else:
            print(f"[OK] {cid} quote verified")

    return ok


In [143]:
result4 = answer_with_citations("What is the governing law of this agreement?", top_k=7)

print("ANSWER:", result4["answer"])
print("CITATIONS:", result4["citations"])

verified = verify_citations(result4)
print("VERIFIED:", verified)

# If it fails, print raw output to see what Gemini returned
if not verified:
    print("\nRAW MODEL OUTPUT:\n", result4["raw_model_output"])


[OK] 7e729bc4b31d5c25 quote verified
ANSWER: This Agreement is governed by English law, and the parties agree to the exclusive jurisdiction of the English courts for any disputes (contractual or non-contractual) related to this Agreement.
CITATIONS: [{'chunk_id': '7e729bc4b31d5c25', 'quote': '11.10 Governing Law. This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights. If this Agreement is translated into any other language, if there is  conflict the English text will take precedence.'}]
[OK] 7e729bc4b31d5c25 quote verified
VERIFIED: True


### FastAPI-ready wrapper

In [162]:
from datetime import datetime, timezone

def rag_answer(question: str, top_k: int = 7, debug: bool = False, doc_id: str | None = None):
    """
    API-ready wrapper.
    Returns a clean dict that you can return from FastAPI as JSON.

    debug=False: hides raw_model_output + retrieved_chunk_ids
    debug=True: includes them (useful during development)
    """
    r = answer_with_citations(question, top_k=top_k, doc_id = doc_id)

    out = {
        "question": r["question"],
        "answer": r["answer"],
        "citations": r["citations"],   # [{chunk_id, quote}, ...]
        "doc_id": doc_id,
        "timestamp_utc": datetime.now(timezone.utc).isoformat(),
    }

    if debug:
        out["retrieved_chunk_ids"] = r.get("retrieved_chunk_ids", [])
        out["raw_model_output"] = r.get("raw_model_output", "")

    return out


In [163]:
resp = rag_answer("What is the governing law of this agreement?", top_k=7, debug=False)
print(resp)


[BAD] quote not found verbatim in chunk 7e729bc4b31d5c25
quote: 11.10 Governing Law. This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an injunction or other relief to protect its Intellectual Property Rights.
{'question': 'What is the governing law of this agreement?', 'answer': "I can't verify the evidence in the provided excerpts.", 'citations': [], 'doc_id': None, 'timestamp_utc': '2026-01-05T22:47:00.957945+00:00'}


In [151]:
with engine.connect() as conn:
    rows_id = conn.execute(text("SELECT doc_id FROM documents")).fetchmany(2)
for r in rows_id:
    print(r)

('2d790a4a4132cc53',)
('43ab152a17a15599',)


In [156]:
id_test = rows_id[0][0]
id_test

'2d790a4a4132cc53'

In [155]:
resp = rag_answer("What is the governing law of this agreement?", top_k=7, debug=False, doc_id = rows_id[0][0])
print(resp)

[OK] 415078aedb15129d quote verified
{'question': 'What is the governing law of this agreement?', 'answer': 'This Agreement is to be governed and interpreted according to the laws of the State of Kansas.', 'citations': [{'chunk_id': '415078aedb15129d', 'quote': 'This Agreement shall be governed and construed according to the laws of the State of Kansas.'}], 'timestamp_utc': '2026-01-05T22:30:23.499286+00:00'}


In [157]:
r_scoped = answer_with_citations(
    "What is the governing law of this agreement?",
    top_k=7,
    doc_id = id_test
)
print("ANSWER:", r_scoped["answer"])
print("CITATIONS:", r_scoped["citations"])
print("RETRIEVED:", r_scoped["retrieved_chunk_ids"])


[OK] 415078aedb15129d quote verified
ANSWER: This Agreement shall be governed and construed according to the laws of the State of Kansas.
CITATIONS: [{'chunk_id': '415078aedb15129d', 'quote': 'This Agreement shall be governed and construed according to the laws of the State of Kansas.'}]
RETRIEVED: ['415078aedb15129d', '094abd6c9b46b708', '917cca69c657ad5a', '6e3be1855d87c17b', 'eaffcc11ce135da8', 'a5686b63584985a7', '71cd33b3003bdcdf']


In [158]:
ids = r_scoped["retrieved_chunk_ids"]

if ids:
    with engine.connect() as conn:
        params = {f"id{i}": cid for i, cid in enumerate(ids)}
        placeholders = ", ".join([f":id{i}" for i in range(len(ids))])

        q = f"""
            SELECT DISTINCT doc_id
            FROM chunks
            WHERE chunk_id IN ({placeholders})
        """
        doc_ids = [x[0] for x in conn.execute(text(q), params).fetchall()]

    print("distinct doc_ids for retrieved chunks:", doc_ids)
else:
    print("No retrieved chunk ids (top_k returned nothing)")


distinct doc_ids for retrieved chunks: ['2d790a4a4132cc53']


In [161]:
doc_ids[0] == id_test

True

In [165]:
resp = rag_answer(
    "What is the governing law of this agreement?",
    doc_id=id_test,
    debug=False,
)
print(resp["doc_id"], resp["answer"])
print(resp["citations"])


[OK] 415078aedb15129d quote verified
2d790a4a4132cc53 This Agreement is governed and shall be construed according to the laws of the State of Kansas.
[{'chunk_id': '415078aedb15129d', 'quote': 'This Agreement shall be governed and construed according to the laws of the State of Kansas.'}]


In [167]:
def list_documents(limit: int = 50, offset: int = 0):
    """
    Returns a list of {doc_id, title, source} for UI dropdowns.
    Offset is used for pagination (each page gives a specified number of documents before you go to next page)
    """
    with engine.connect() as conn:
        rows = conn.execute(text("""
            SELECT doc_id, title, source
            FROM documents
            ORDER BY title
            LIMIT :limit OFFSET :offset
        """), {"limit": limit, "offset": offset}).fetchall()

    out = []
    for r in rows:
        out.append({"doc_id": r[0], "title": r[1], "source": r[2]})
    return out


In [168]:
list_documents()

[{'doc_id': '2d790a4a4132cc53',
  'title': 'ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT AGREEMENT',
  'source': 'cuad-qa'},
 {'doc_id': 'b9d11c50bada2ade',
  'title': 'KIROMICBIOPHARMA,INC_05_11_2020-EX-10.23-CONSULTING AGREEMENT',
  'source': 'cuad-qa'},
 {'doc_id': 'f71b4149a185d016',
  'title': 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT',
  'source': 'cuad-qa'},
 {'doc_id': '644b67c819fbca9c',
  'title': 'NELNETINC_04_08_2020-EX-1-JOINT FILING AGREEMENT',
  'source': 'cuad-qa'},
 {'doc_id': 'b26c38227ce11a4e',
  'title': 'NeuroboPharmaceuticalsInc_20190903_S-4_EX-10.36_11802165_EX-10.36_Manufacturing Agreement_ Supply Agreement',
  'source': 'cuad-qa'},
 {'doc_id': '43ab152a17a15599',
  'title': 'WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION AND DISTRIBUTION AGREEMENT',
  'source': 'cuad-qa'}]

In [171]:
def get_document(doc_id: str):
    """
    Returns one document row: {doc_id, title, source}.
    """
    with engine.connect() as conn:
        r = conn.execute(text("""
            SELECT doc_id, title, source
            FROM documents
            WHERE doc_id = :doc_id
        """), {"doc_id": doc_id}).fetchone()

    if r is None:
        return None

    return {"doc_id": r[0], "title": r[1], "source": r[2]}


In [172]:
docs = list_documents(limit=10)
print("docs:", len(docs))
for d in docs:
    print(d["doc_id"], "-", d["title"][:70])

selected = docs[0]["doc_id"]
print("\nSelected doc:", get_document(selected))

resp = rag_answer("What is the governing law of this agreement?", doc_id=selected, top_k=7)
print("\nAnswer:", resp["answer"])
print("Citations:", resp["citations"])


docs: 6
2d790a4a4132cc53 - ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT AGREEMENT
b9d11c50bada2ade - KIROMICBIOPHARMA,INC_05_11_2020-EX-10.23-CONSULTING AGREEMENT
f71b4149a185d016 - LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT
644b67c819fbca9c - NELNETINC_04_08_2020-EX-1-JOINT FILING AGREEMENT
b26c38227ce11a4e - NeuroboPharmaceuticalsInc_20190903_S-4_EX-10.36_11802165_EX-10.36_Manu
43ab152a17a15599 - WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION AND DISTRIBUTION AGREEMEN

Selected doc: {'doc_id': '2d790a4a4132cc53', 'title': 'ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT AGREEMENT', 'source': 'cuad-qa'}
[OK] 415078aedb15129d quote verified

Answer: This Agreement shall be governed and construed according to the laws of the State of Kansas.
Citations: [{'chunk_id': '415078aedb15129d', 'quote': 'This Agreement shall be governed and construed according to the laws of the State of Kansas.'}]


### FastAPI

In [173]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class ChatRequest(BaseModel):
    doc_id: str
    question: str
    top_k: int = 7
    debug: bool = False

@app.get("/documents")
def api_list_documents(limit: int = 50, offset: int = 0):
    return list_documents(limit=limit, offset=offset)

@app.post("/chat")
def api_chat(req: ChatRequest):
    return rag_answer(
        req.question,
        doc_id=req.doc_id,
        top_k=req.top_k,
        debug=req.debug,
    )


In [174]:

def list_documents(limit: int = 50, offset: int = 0, q: str | None = None):
    """
    Returns {doc_id, title, source} rows.
    If q is provided, filters by title substring (case-insensitive-ish for SQLite).
    """
    base_sql = """
        SELECT doc_id, title, source
        FROM documents
    """

    params = {"limit": limit, "offset": offset}

    if q is not None and q.strip() != "":
        # SQLite LIKE is case-insensitive for ASCII by default in many setups,
        # but not guaranteed for all locales. Good enough for now.
        base_sql += " WHERE title LIKE :q"
        params["q"] = f"%{q.strip()}%"

    base_sql += " ORDER BY title LIMIT :limit OFFSET :offset"

    with engine.connect() as conn:
        rows = conn.execute(text(base_sql), params).fetchall()

    out = []
    for r in rows:
        out.append({"doc_id": r[0], "title": r[1], "source": r[2]})
    return out


In [175]:
hits = list_documents(limit=10, offset=0, q="ENDORSEMENT")
print("hits:", len(hits))
for h in hits:
    print(h["doc_id"], "-", h["title"][:80])


hits: 1
2d790a4a4132cc53 - ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT AGREEMENT
